# Packages 

In [1]:
import boto3
import sagemaker
from pyathena import connect

import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set_style = "seaborn-whitegrid"

sns.set(
    rc={
        "font.style": "normal",
        "axes.facecolor": "white",
        "grid.color": ".8",
        "grid.linestyle": "-",
        "figure.facecolor": "white",
        "figure.titlesize": 20,
        "text.color": "black",
        "xtick.color": "black",
        "ytick.color": "black",
        "axes.labelcolor": "black",
        "axes.grid": True,
        "axes.labelsize": 10,
        "xtick.labelsize": 10,
        "font.size": 10,
        "ytick.labelsize": 10,
    }
)

# Set-up

In [2]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [3]:
ingest_create_athena_db_passed = False

# Create Athena Database

In [4]:
# Set Athena database & table
database_name = "ads508"
table_name = "flight_departure_delays"

In [5]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/ads508/athena/staging".format(bucket)

In [6]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [7]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS ads508


In [8]:
pd.read_sql(statement, conn)

""


# Verify The Database Has Been Created Succesfully

In [9]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,ads508
1,default
2,dsoaws


In [10]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [11]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


# Download Data from Public S3 Bucket

In [12]:
# Public Flight Data
s3_client = boto3.client("s3")

BUCKET='ads-508-airline'
KEY='transformed/ON_TIME_REPORTING_12.csv'

response = s3_client.get_object(Bucket=BUCKET, Key=KEY)
dec_flight = pd.read_csv(response.get("Body"))
dec_flight.head()

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,ORIGIN,DEST,DEP_DEL15,DEP_TIME_BLK,ARR_TIME_BLK,CANCELLED,CRS_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,8,7,WN,N8651A,STL,SAN,0.0,1100-1159,1300-1359,0.0,245.0,1557.0,7,0.0,0.0,18.0,0.0,0.0
1,8,7,WN,N939WN,STL,SAT,0.0,1200-1259,1400-1459,0.0,145.0,786.0,4,NaN,NaN,NaN,NaN,NaN
2,8,7,WN,N7741C,STL,SAT,0.0,2100-2159,0001-0559,0.0,140.0,786.0,4,NaN,NaN,NaN,NaN,NaN
3,8,7,WN,N550WN,STL,SEA,0.0,0900-0959,1200-1259,0.0,275.0,1709.0,7,NaN,NaN,NaN,NaN,NaN
4,8,7,WN,N8319F,STL,SFO,1.0,1800-1859,2000-2059,0.0,270.0,1735.0,7,NaN,NaN,NaN,NaN,NaN


In [13]:
file_path = Path('../src/data/transformed_data/ON_TIME_REPORTING_12.csv')

if file_path.is_file():
    None
else:
    dec_flight.to_csv('../src/data/transformed_data/ON_TIME_REPORTING_12.csv', index=False)

# Set S3 Destination Location(Our S3 Private Bucket)

In [14]:
s3_private_path_csv = "s3://{}/ads508/data".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-229768475194/ads508/data


In [15]:
%store s3_private_path_csv

Stored 's3_private_path_csv' (str)


# Copy Downloaded Local Data to our Private S3 Bucket in this Account

In [16]:
!aws s3 cp $file_path $s3_private_path_csv/ 

upload: ../src/data/transformed_data/ON_TIME_REPORTING_12.csv to s3://sagemaker-us-east-1-229768475194/ads508/data/ON_TIME_REPORTING_12.csv


In [17]:
!aws s3 ls $s3_private_path_csv/

2022-03-21 17:49:04   45671905 ON_TIME_REPORTING_12.csv


# Create Table in Database

In [18]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         DAY_OF_MONTH int,
         DAY_OF_WEEK int,
         OP_UNIQUE_CARRIER string,
         TAIL_NUM string,
         ORIGIN string,
         DEST string,
         DEP_DEL15 float,
         DEP_TIME_BLK string,
         ARR_TIME_BLK string,
         CANCELLED float,
         CRS_ELAPSED_TIME float,
         DISTANCE float,
         DISTANCE_GROUP int,
         CARRIER_DELAY float,
         WEATHER_DELAY float,
         NAS_DELAY float,
         SECURITY_DELAY float,
         LATE_AIRCRAFT_DELAY float
     
         
        
) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name, s3_private_path_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS ads508.flight_departure_delays(
         DAY_OF_MONTH int,
         DAY_OF_WEEK int,
         OP_UNIQUE_CARRIER string,
         TAIL_NUM string,
         ORIGIN string,
         DEST string,
         DEP_DEL15 float,
         DEP_TIME_BLK string,
         ARR_TIME_BLK string,
         CANCELLED float,
         CRS_ELAPSED_TIME float,
         DISTANCE float,
         DISTANCE_GROUP int,
         CARRIER_DELAY float,
         WEATHER_DELAY float,
         NAS_DELAY float,
         SECURITY_DELAY float,
         LATE_AIRCRAFT_DELAY float
     
         
        
) 
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
LINES TERMINATED BY '
'
LOCATION 's3://sagemaker-us-east-1-229768475194/ads508/data'
TBLPROPERTIES ('skip.header.line.count'='1')


In [19]:
# Droping table if needed
#statement2 = "DROP TABLE {}.{}".format(database_name, table_name)
#pd.read_sql(statement2, conn)

In [20]:
pd.read_sql(statement, conn)

""


# Verify the table has been created successfully

In [21]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,tab_name
0,flight_departure_delays


# Run a sample query 

In [22]:
statement = """SELECT * FROM {}.{}
                LIMIT 10""".format(
    database_name, table_name
)

print(statement)

SELECT * FROM ads508.flight_departure_delays
                LIMIT 10


In [23]:
pd.read_sql(statement, conn)

,day_of_month,day_of_week,op_unique_carrier,tail_num,origin,dest,dep_del15,dep_time_blk,arr_time_blk,cancelled,crs_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,4,3,WN,N738CB,MDW,OMA,1.0,2200-2259,2300-2359,0.0,85.0,423.0,2,1.0,0.0,0.0,0.0,32.0
1,4,3,WN,N7869A,MDW,ONT,1.0,1900-1959,2100-2159,0.0,260.0,1706.0,7,17.0,0.0,0.0,0.0,0.0
2,4,3,WN,N938WN,MDW,ORF,0.0,2100-2159,0001-0559,0.0,115.0,704.0,3,NaN,NaN,NaN,NaN,NaN
3,4,3,WN,N902WN,MDW,ORF,1.0,1300-1359,1600-1659,0.0,125.0,704.0,3,NaN,NaN,NaN,NaN,NaN
4,4,3,WN,N8584Z,MDW,PDX,0.0,0800-0859,1100-1159,0.0,280.0,1751.0,8,NaN,NaN,NaN,NaN,NaN
5,4,3,WN,N744SW,MDW,PHL,0.0,0001-0559,0800-0859,0.0,110.0,668.0,3,NaN,NaN,NaN,NaN,NaN
6,4,3,WN,N8501V,MDW,PHL,1.0,2100-2159,0001-0559,0.0,105.0,668.0,3,19.0,0.0,0.0,0.0,0.0
7,4,3,WN,N776WN,MDW,PHL,0.0,1500-1559,1800-1859,0.0,120.0,668.0,3,NaN,NaN,NaN,NaN,NaN
8,4,3,WN,N423WN,MDW,PHL,1.0,1100-1159,1400-1459,0.0,115.0,668.0,3,18.0,0.0,0.0,0.0,0.0
9,4,3,WN,N720WN,MDW,PHL,NaN,0900-0959,1100-1159,1.0,110.0,668.0,3,NaN,NaN,NaN,NaN,NaN


# Review the New Athena Table in the Glue Catalog

In [24]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)

----
## 1. What days of the month are best and worst for departure delays?

In [25]:
statement = """WITH counts AS (SELECT day_of_month,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM {}.{}
                GROUP BY day_of_month
                ORDER BY day_of_month),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                """.format(
    database_name, table_name
)

print(statement)

WITH counts AS (SELECT day_of_month,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM ads508.flight_departure_delays
                GROUP BY day_of_month
                ORDER BY day_of_month),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                


In [26]:
pd.read_sql(statement, conn)

,day_of_month,on_time,delayed,total,percent_delayed
0,7,14303,1732,16035,0.108014
1,25,14579,1884,16463,0.114438
2,8,17762,2459,20221,0.121606
3,5,18275,2906,21181,0.137198
4,6,18070,3116,21186,0.147078
5,10,16760,2909,19669,0.147898
6,15,17126,3057,20183,0.151464
7,12,17992,3217,21209,0.151681
8,24,13902,2769,16671,0.166097
9,14,13307,2824,16131,0.175067


**Summary:**<br>
Days of the month show differences, but no obvious pattern - let's explore day of the week.

## 2. What day of the week is the best and worst for depature delays?

In [27]:
statement = """WITH counts AS (SELECT day_of_week,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM {}.{}
                GROUP BY day_of_week
                ORDER BY day_of_week),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                """.format(
    database_name, table_name
)

print(statement)

WITH counts AS (SELECT day_of_week,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM ads508.flight_departure_delays
                GROUP BY day_of_week
                ORDER BY day_of_week),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                


In [28]:
pd.read_sql(statement, conn)

,day_of_week,on_time,delayed,total,percent_delayed
0,4,69481,15126,84607,0.178780
1,5,68868,15807,84675,0.186678
2,3,62131,14977,77108,0.194234
3,2,74760,18449,93209,0.197932
4,6,57531,14217,71748,0.198152
5,7,80845,23404,104249,0.224501
6,1,78480,26177,104657,0.250122


**Summary:**<br>
The best days of the week are 4 (Thursday) @ 17.9% and 5 (Friday) @ 18.7%.<br>
The worst days of the week are 1 (Monday) @ 25% and 7 (Sunday) @ 22.5%.<br>

## 3. What distance groups perform best and worst for departure delays?

In [29]:
statement = """WITH counts AS (SELECT distance_group,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM {}.{}
                GROUP BY distance_group
                ORDER BY distance_group),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                """.format(
    database_name, table_name
)

print(statement)

WITH counts AS (SELECT distance_group,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM ads508.flight_departure_delays
                GROUP BY distance_group
                ORDER BY distance_group),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                


In [30]:
pd.read_sql(statement, conn)

,distance_group,on_time,delayed,total,percent_delayed
0,1,63452,14152,77604,0.182362
1,3,98656,24926,123582,0.201696
2,4,77200,20107,97307,0.206635
3,2,117550,30683,148233,0.206992
4,6,21412,5642,27054,0.208546
5,10,12621,3554,16175,0.219722
6,5,54398,15357,69755,0.220156
7,7,20416,5856,26272,0.222899
8,9,7205,2086,9291,0.224518
9,11,8931,2640,11571,0.228157


**Summary:**<br>
The best distance groups are 1 (<250 miles) @ 18.2% and 3 (500-749 miles) @ 21.2%.<br>
The worst distance groups are 8 (1750-1999 Miles) @ 23.5% and 11 (>2500 Miles) @ 23.5%.<br>

## 4. Number of Unique Values 

In [31]:
statement = """SELECT COUNT(DISTINCT op_unique_carrier) as op_unique_carrier,
               COUNT(DISTINCT tail_num) as tail_num,
               COUNT(DISTINCT origin) as origin,
               COUNT(DISTINCT dest) as dest,
               COUNT(DISTINCT dep_time_blk) as dep_time_blk,
               COUNT (DISTINCT arr_time_blk) as arr_time_blk
            FROM {}.{}
            
            """.format(
    database_name, table_name
)

print(statement)

SELECT COUNT(DISTINCT op_unique_carrier) as op_unique_carrier,
               COUNT(DISTINCT tail_num) as tail_num,
               COUNT(DISTINCT origin) as origin,
               COUNT(DISTINCT dest) as dest,
               COUNT(DISTINCT dep_time_blk) as dep_time_blk,
               COUNT (DISTINCT arr_time_blk) as arr_time_blk
            FROM ads508.flight_departure_delays
            
            


In [32]:
pd.read_sql(statement, conn)

,op_unique_carrier,tail_num,origin,dest,dep_time_blk,arr_time_blk
0,17,5479,350,350,19,19


**NOTE:**  
High cardinality in Tail_Num, Origin, Dest make analysis difficult.

## 5. What are the best and worst performing airlines for departure delays?

In [33]:
statement = """WITH counts AS (SELECT op_unique_carrier,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM {}.{}
                GROUP BY op_unique_carrier
                ORDER BY op_unique_carrier),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                """.format(
    database_name, table_name
)

print(statement)

WITH counts AS (SELECT op_unique_carrier,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM ads508.flight_departure_delays
                GROUP BY op_unique_carrier
                ORDER BY op_unique_carrier),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                


In [34]:
pd.read_sql(statement, conn)

,op_unique_carrier,on_time,delayed,total,percent_delayed
0,HA,6618,651,7269,0.089558
1,DL,68764,12736,81500,0.156270
2,9E,19174,3960,23134,0.171177
3,AA,65242,14001,79243,0.176684
4,MQ,21869,4877,26746,0.182345
5,NK,14064,3146,17210,0.182801
6,YX,23081,5263,28344,0.185683
7,UA,41276,9889,51165,0.193277
8,OO,56526,14016,70542,0.198690
9,OH,18804,5218,24022,0.217218


**Summary:**<br>
We note a wide range of percent of departures delayed by carrier. This could indicate that carrier-specific data (such as staffing) could be good indicators for predicting delays.<br>

Mean % Delayed departures = 20.7% <br>
Worst performaing carriers = B6 (JetBlue) @ 31.3% and WN (Southwest) @ 26.1%<br>
Best performing carriers = HA (Hawaiian Airlines) @ 9% and DL (Delta Airlines) @ 15.6%<br>

## 6. What are the best and worst performing time blocks for departure delays?

In [35]:
statement = """WITH counts AS (SELECT dep_time_blk,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM {}.{}
                GROUP BY dep_time_blk
                ORDER BY dep_time_blk),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                """.format(
    database_name, table_name
)

print(statement)

WITH counts AS (SELECT dep_time_blk,
                sum(case DEP_DEL15 when 0 then 1 end) on_time,
                sum(case DEP_DEL15 when 1 then 1 end) delayed
                FROM ads508.flight_departure_delays
                GROUP BY dep_time_blk
                ORDER BY dep_time_blk),
                
                total_sum as (SELECT *,  (on_time + delayed) as total
                FROM counts)
                
                SELECT *, 
                delayed / CAST(total AS double) as percent_delayed  
                FROM total_sum
                ORDER BY percent_delayed
                


In [36]:
pd.read_sql(statement, conn)

,dep_time_blk,on_time,delayed,total,percent_delayed
0,0600-0659,42566,3952,46518,0.084956
1,0001-0559,17565,1964,19529,0.100568
2,0700-0759,36653,4376,41029,0.106656
3,0800-0859,34384,5254,39638,0.132550
4,0900-0959,30593,5706,36299,0.157194
5,1000-1059,30423,6876,37299,0.184348
6,1100-1159,30424,7374,37798,0.195090
7,1200-1259,30721,8133,38854,0.209322
8,1300-1359,27005,7768,34773,0.223392
9,2300-2359,3738,1098,4836,0.227047


**Answer:**<br>
Best times = 6-659am @ 8.5% and 1201am - 6am @ 10%<br>
Worst times = 7-759pm @ 29.9% and 9-959pm @ 29.7%<br>

# Store Variables for the Next Notebooks

In [37]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-10-22-14-04'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:229768475194:endpoint
autopilot_endpoint_name                               -> 'automl-dm-ep-10-22-58-33'
autopilot_model_arn                                   -> 'arn:aws:sagemaker:us-east-1:229768475194:model/au
autopilot_model_name                                  -> 'automl-dm-model-10-22-58-32'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-229768475194/data/amazon
comprehend_train_s3_uri                               -> 's3://sagemaker-us-east-1-229768475194/data/amazon
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
s3_private_path_csv                                   -> 's3://sagemaker-us-east-1-229768

# Release Resources

In [38]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [39]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>